In [1]:
# imports 

import numpy as np
import heapq
import math
import scipy as sp
from matplotlib import pyplot
from typing import NamedTuple

In [ ]:
#################### TODO ####################

# We can copy and paste the main code and change policies and metrics

# Simple model is done so test different metrics
# Work on more "realisitc" model

In [2]:
#################### PARAMETERS ####################

# m         : the number of floors in the building
# n         : the number of elevators 
# lambdas   : m * m array; i,j is the rate of arrivals on floor i who want to go to floor j
# occupancy : max occupancy of an elevator
# h         : length of simulation

#################### SIMULATION DESIGN ####################

# Elevators have predetermined paths
# On button press, scheduler decides which paths change
# When elevator reaches a floor people get off 
# Arrivals are generated retrospectively with a queue to store excess

# Button timings are events on clock and so are elevator arrivals

#################### NOTES ####################

# Floors are labeled 0 ... m - 1
# Assume time to travel between floors is constant
# We'll assume loading happens in 0 time i.e. instantly

# All 2m buttons are initialized to begin 
# then buttons are pressed after elevator leaves the floor

#################### DATA STRUCTURES ####################

# Feel free to modify, but provide documentation

# down  : tuple where index j is rate of clients of floor j going down
# total : tuple where index j is sum of all rates

# elevators: list of elevator structures

# elevator structure: clients    : list where index j is amount of people want to go to floor j
#                     path       : lists of pairs (floor number, time of arrival to floor)
#                     floor      : the current floor
#                     direction  : 1 for up, 0 for stationary, -1 for down
#                     size       : amount of people inside

# floors: list of floor structures

# floor structure: clients : list where index j is amount of people want to go to floor j
#                  button  : list of pairs (floor, time of press, direction)
#                  time    : pair of last times that elevator came to service up/down

#################### Dictionary ####################
# c  |-> closest
# e  |-> elevator
# ix |-> index
# t  |-> time
# b  |-> button

def simple_sim(m: int, n: int, lambdas: tuple[tuple[float]], occupancy: int, h: int):
    # Elevator structure
    class Elevator(NamedTuple):
        clients: list
        path: list
        floor: int
        direction: int
    
    # Floor structure
    class Floor(NamedTuple):
        clients: list
        times: list

    # Initialization 
    rng       = np.random.default_rng()
    t         = 0  # total elapsed time
    e_clocks  = [] # clocks for elevator arrivals
    b_clocks  = [] # clocks for the button presses
    floors    = list( Floor([], []) for _ in range(m) )
    elevators = list( Elevator(np.zeros(m), [], 0) for _ in range(n) )
    down      = tuple( sum(lambdas[j][:j]) for j in range(m) )
    total     = tuple( sum(lambdas[j]) for j in range(m) )

    # initialize with both up and down presses
    # fix floors[i].times
    for i in range(m):
        first = [i, rng.exponential(1/total[i])] # time of press
        second = [i]
        # type of press
        if rng.uniform(0, 1) <= down[i] / total[i]:
            first.append(-1)
            second.append(first[0] + rng.exponential(1/(total[i] - down[i])))
            second.append(1)
            floors[i].times
        else:
            first.append(1)
            second.append(first[0] + rng.exponential(1/down[i]))
            second.append(-1)
        clocks.append(tuple(first))
        clocks.append(tuple(second))
        #floors[i].append(pair) IDK
    heapq.heapify(clocks)

    # Finite horizon
    while(t <= h):

        # Determine if scheduler or elevator section
        if e_clocks == [] or b_clocks[0][1] <= e_clocks[0][1]:
        # SCHEDULER SECTION

            floor, time, direction = heapq.heappop(b_clocks)

            # We need to choose the "best" elevator that will have path modified
            # This is where we can test different policies
            # for now policy will be "closest" elevator
            # the elvator that has a scheduled stop closest to the floor rung
            # ties will be broken by earliest time of arrival to that floor

            # The algorithm below "streams" in the elevators and keeps a running "best" one
            c_distance = floor    # closest distance from a scheduled floor to rung floor
            c_e_arrival_t = -1    # the time of arrival to rung floor by closest elevator
            e_ix = -1             # index of elevator to have path modified
            path_ix = -1          # the index to insert into the path
            for elevator, i in enumerate(elevators):
                if elevator.direction == direction: 
                    # Search through its path to see if its a good candidate
                    for j, e_floor, e_finish_t, j in enumerate(elevator.path):
                        if (direction == 1 and 0 <= floor - e_floor <= c_distance) or 
                        (direction == -1 and 0 <= e_floor - floor <= c_distance):
                            c_distance = np.abs(e_floor - floor)
                            if e_ix == -1 or e_finish_t + c_distance <= c_e_arrival_t:
                                c_e_arrival_t = e_finish_t + c_distance # takes one unit of time to move a floor
                                e_ix = i
                                path_ix = j
                        else:
                            break
                elif elevator.direction == 0:
                    if np.abs(elevator.floor - floor) <= c_distance:
                        c_distance = np.abs(elevator.floor - floor)
                        if e_ix == -1 or c_distance <= c_e_arrival_t:
                            c_e_arrival_t = c_distance # takes one unit of time to move a floor
                            e_ix = i
                            path_ix = 0

            # variables now store "closest" path information so we augment the relevant path
            elevators[i].path.insert(j + 1, (floor, c_e_arrival_t))
            if elevators[i].direction == 0:
                elevators[i].direction = 1 if floor - elevators[i].floor >= 0 else -1
        # ELEVATOR SECTION
        else:
            floor, time, i = heapq.heappop(e_clocks)
            # Unload people
            elevators[i].capacity -= elevators[i].clients[floor]
            elevators[i].clients[floor] = 0
            # Load people
            for clients, i in enumerate(floors[i].clients):
                












    return # performance metrics

In [9]:
l= [[1, 2], [3, 5], [10, 100]]
l.insert(1, [2,3])
for i in enumerate(l):
    print(i)

(0, [1, 2])
(1, [2, 3])
(2, [3, 5])
(3, [10, 100])


In [4]:
#tracking the clocks of button change of every floor
def DES_engine(m: int, lambda_param, h: int, buckets):
    rng = np.random.default_rng() # rv generator
    t = 0.0
    n = 0
    generate_tuples = lambda m: [(0, 0) for _ in range(m)]
    generate_clocks = lambda m: [rng.exponential(1/lambda_param) for _ in range(m)]
    floor_states = generate_tuples(m) #initial state of every floor is (0,0)
    button_arrival = generate_clocks(m) #generates arrival times for each button on each floor, maybe have to make it a tuple for up and down ?
    q_up = 0
    q_down = 0
    
    SOF = [button_arrival, floor_states, q_up, q_down]
    
    while (t<h):
        next = min(button_arrival)
        tau = next[0]
        floor = button_arrival.index(tau)
        for i in buckets:
            if i < up:
                